### Scratchwork for PEC Development

Workflow:
Input_VEDA + Input_Reference -> Output_Calculation

Idea: one instance per calculation run
Relations: Output_Calculation maps to a Input_VEDA + Input_Reference

*VEDA Input: either an API or hard-passed URL to a VEDA dataset*
- API: accessed through VEDA API (priority)
- URL: directly passed for access (secondary due to complications)
- Class
- Global vars:
    - Live track of all instances
- Instance vars:
    - Hash?
    - CRS
    - API or URL
    - Bounding box
    - Time
- Methods: 
    - getInstanceDetails()
    - 

*Reference Input: generalized input which we compare VEDA input to*
- Class
- Global vars:
    - Live tracking
    - 
- Instance vars:
    - Hash?
    - CRS
    - URL/location
    - Platform/how to read
- Methods:
    - Read_from_source()
    - getInstanceDetails()

*Calculation Output: generalized output containing all calculations + metadata/settings to replicate*
- Class
- Global vars:
    - live_track_all
- Instance vars:
    - what_calculations
    - meta_data
- Methods:
    - plot/visualize()
    - get_metadata()
    - dump_contents()
    - regen_from_contents()

Back-burner:
- Able to load in previous instances by dumping contents to Output_Calculation?
    - For plotting, re-analysis/re-generation, etc?
- Do we want one ipynb run per results? Multi-tracking?
    - Maybe not multiple instances, but we want to neatly track properties
- Any subclasses? Private/Public
 
 
 
 #######################

# Test VEDA API Access for FirePerims

In [3]:
!pip install OWSLib==0.28.1 --quiet

In [55]:
from owslib.ogcapi.features import Features
import geopandas as gpd
import datetime as dt
from datetime import datetime, timedelta

In [56]:
OGC_URL = "https://firenrt.delta-backend.com"

w = Features(url=OGC_URL)
w.feature_collections()

['public.eis_fire_lf_perimeter_nrt',
 'public.eis_fire_lf_newfirepix_nrt',
 'public.eis_fire_lf_fireline_archive',
 'public.eis_fire_lf_perimeter_archive',
 'public.eis_fire_lf_nfplist_nrt',
 'public.eis_fire_lf_fireline_nrt',
 'public.eis_fire_snapshot_newfirepix_nrt',
 'public.eis_fire_lf_nfplist_archive',
 'public.eis_fire_snapshot_fireline_nrt',
 'public.eis_fire_lf_newfirepix_archive',
 'public.eis_fire_fwi',
 'public.eis_fire_snapshot_perimeter_nrt',
 'public.st_squaregrid',
 'public.st_hexagongrid',
 'public.st_subdivide']

In [57]:
perm = w.collection("public.eis_fire_snapshot_perimeter_nrt")
print(type(perm))
print(perm)

<class 'dict'>
{'id': 'public.eis_fire_snapshot_perimeter_nrt', 'title': 'public.eis_fire_snapshot_perimeter_nrt', 'links': [{'href': 'https://firenrt.delta-backend.com/collections/public.eis_fire_snapshot_perimeter_nrt', 'rel': 'self', 'type': 'application/json'}, {'href': 'https://firenrt.delta-backend.com/collections/public.eis_fire_snapshot_perimeter_nrt/items', 'rel': 'items', 'type': 'application/geo+json', 'title': 'Items'}, {'href': 'https://firenrt.delta-backend.com/collections/public.eis_fire_snapshot_perimeter_nrt/items?f=csv', 'rel': 'alternate', 'type': 'text/csv', 'title': 'Items (CSV)'}, {'href': 'https://firenrt.delta-backend.com/collections/public.eis_fire_snapshot_perimeter_nrt/items?f=geojsonseq', 'rel': 'alternate', 'type': 'application/geo+json-seq', 'title': 'Items (GeoJSONSeq)'}, {'href': 'https://firenrt.delta-backend.com/collections/public.eis_fire_snapshot_perimeter_nrt/queryables', 'rel': 'queryables', 'type': 'application/schema+json', 'title': 'Queryables'}

In [58]:
perm["extent"]["temporal"]["interval"][0]

['2023-09-03T00:00:00+00:00', '2023-09-23T00:00:00+00:00']

In [59]:
most_recent_time = max(*perm["extent"]["temporal"]["interval"])
print(type(most_recent_time))
most_recent_time = max(*perm["extent"]["temporal"]["interval"])
now = dt.datetime.strptime(most_recent_time, "%Y-%m-%dT%H:%M:%S+00:00")
last_week = now - dt.timedelta(weeks=1)
last_week = dt.datetime.strftime(last_week, "%Y-%m-%dT%H:%M:%S+00:00")
print("Most Recent Time =", most_recent_time)
print("Last week =", last_week)

<class 'str'>
Most Recent Time = 2023-09-23T00:00:00+00:00
Last week = 2023-09-16T00:00:00+00:00


In [60]:
perm_results = w.collection_items(
    "public.eis_fire_snapshot_perimeter_nrt",  # name of the dataset we want
    bbox=["-106.8", "24.5", "-72.9", "37.3"],  # coodrinates of bounding box,
    datetime=[last_week + "/" + most_recent_time],  # date range
    limit=1000,  # max number of items returned
    filter="farea>5 AND duration>2",  # additional filters based on queryable fields
)
# perm_results
print(last_week)
print(most_recent_time)

perm_results = w.collection_items(
    "public.eis_fire_snapshot_perimeter_nrt",  # name of the dataset we want
    bbox=["-106.8", "24.5", "-72.9", "37.3"],  # coodrinates of bounding box,
    datetime=[last_week + "/" + most_recent_time],  # date range
    limit=1000  # max number of items returned  # additional filters based on queryable fields
)

print("public.eis_fire_snapshot_perimeter_nrt",  # name of the dataset we want
      ["-106.8", "24.5", "-72.9", "37.3"],  # coodrinates of bounding box,
     [last_week + "/" + most_recent_time],  # date range
    1000)

2023-09-16T00:00:00+00:00
2023-09-23T00:00:00+00:00
public.eis_fire_snapshot_perimeter_nrt ['-106.8', '24.5', '-72.9', '37.3'] ['2023-09-16T00:00:00+00:00/2023-09-23T00:00:00+00:00'] 1000


In [61]:
df = gpd.GeoDataFrame.from_features(perm_results["features"])
print(type(df))
print(type("hi"))

<class 'geopandas.geodataframe.GeoDataFrame'>
<class 'str'>


# Read NIFC Polygons from Local MAAP

In [66]:
df = gpd.read_file("/projects/shared-buckets/ksharonin/InterAgencyFirePerimeterHistory")
df

,OBJECTID,MAP_METHOD,DATE_CUR,COMMENTS,GEO_ID,IRWINID,UNQE_FIRE_,FIRE_YEAR,LOCAL_NUM,INCIDENT,...,POO_RESP_I,FEATURE_CA,GIS_ACRES,USER_NAME,SOURCE,AGENCY,FIRE_YEAR_,Shape__Are,Shape__Len,geometry
0,1,Image Interpretation,20210903,Perimeter update using Sentinel-2 imagery.,1,{9FABE71D-843F-4624-859C-A7EC4A72AB89},2021-AKUYD-000223,2021,000223,Cultas Creek,...,AKUYD,Wildfire Final Perimeter,40450.000000,None,AICC,AFS,2021.0,9.273137e+08,528166.832442,"POLYGON ((-142.81944 65.16774, -142.81904 65.1..."
1,2,Image Interpretation,20210903,Perimeter updated using Sentinel-2 imagery.,2,{C5ED6535-8217-4130-B795-E81AE6BF02D1},2021-AKUYD-000309,2021,000309,Crescent Creek,...,AKUYD,Wildfire Final Perimeter,1384.000000,None,AICC,AFS,2021.0,3.115264e+07,56968.266410,"POLYGON ((-143.66148 64.97452, -143.65997 64.9..."
2,3,Image Interpretation,20210901,Perimeter updated using Sentinel-2 imagery.,3,{D9F72569-F8CD-45DB-93F7-D7D05C4F4723},2021-AKFAS-000397,2021,000397,Salcha River 2,...,AKFAS,Wildfire Final Perimeter,2828.100000,None,AICC,DOF,2021.0,6.324108e+07,118033.611976,"MULTIPOLYGON (((-145.24616 64.90080, -145.2455..."
3,4,Image Interpretation,20210901,Perimeter updated using Sentinel-2 imagery.,4,{3B2C6FF0-CE08-4960-AC83-45DC5687A090},2021-AKUYD-000405,2021,000405,Whitefish Lake,...,AKUYD,Wildfire Final Perimeter,417.500000,None,AICC,AFS,2021.0,1.036802e+07,16248.978885,"POLYGON ((-142.40339 66.24461, -142.40385 66.2..."
4,5,Image Interpretation,20210901,Perimeter updated using Sentinel-2 imagery.,5,{C0CFD381-B561-4704-BA86-311CF4BE126D},2021-AKUYD-000399,2021,000399,South Fork,...,AKUYD,Wildfire Final Perimeter,884.600000,None,AICC,AFS,2021.0,2.040334e+07,44186.061518,"POLYGON ((-144.39133 65.30406, -144.39133 65.3..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125675,125691,GPS-Walked,202105171330,None,{517AAA9D-62F9-4888-8C4D-8C77149A4A13},{74A91A8C-55CE-4C46-A385-87243819C0C1},2021-MTNWS-000286,2021,000286,Coyote,...,MTNWS,Wildfire Daily Fire Perimeter,1.161029,aking_nifc,WFIGS,DNRC,2021.0,1.054388e+04,496.477170,None
125676,125692,Auto-generated,202105272211,Auto-generated by EGP-IRWIN,{15D2ADB9-5502-47BA-A9F5-7CC319BD5FEC},{3FA0C291-950E-4AD9-B127-4C312A8AAA6B},2021-MEMES-21003498,2021,21003498,Lily Pond,...,MEMES,Wildfire Daily Fire Perimeter,0.257079,jcousins_nifc,WFIGS,SFS,2021.0,2.026703e+03,201.291769,None
125677,125693,Mixed Methods,202106211902,None,{19F6EDF6-9E59-4CA5-9E9B-74826AC9F9F7},{9B1E6F33-DBB9-409A-9A8E-07D980611130},2021-CABTU-007844,2021,007844,PARK,...,CABTU,Wildfire Daily Fire Perimeter,399.558800,akirsch_nifc,WFIGS,None,2021.0,2.740989e+06,8463.546256,None
125678,125694,Mixed Methods,202107012143,None,{9EA329B0-5A12-498F-B2A9-EB4334714DB6},{842B03F5-C5EF-45BA-9D4D-9DE8E13F96B2},2021-WYNAX-210210,2021,210210,Lone tree 3,...,WYNAX,Wildfire Daily Fire Perimeter,0.168202,NIFC_Admin,WFIGS,None,2021.0,1.257000e+03,206.304774,None
